In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import configparser
import os
import argparse
import pickle
from function import*
import pybedtools
from pybedtools import BedTool
import shutil 

2022-04-23 03:46:31.449221: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu8/3.1.4/lib:/opt/ohpc/pub/compiler/gcc/8.3.0/lib64:/usr/local/cuda/lib64
2022-04-23 03:46:31.449251: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
def main_pip(sequence,model, chrom=None, start = None, end =None, pamCoord = None, transcript = None, gene=None, cell_line=None):
#     args = Args()
    print("checking sequence input...")
    onehot, bplength = seq(sequence)
     
    chrom = [chrom]
    start = [start]
    end = [end]
    pos = pd.DataFrame(columns = ['chrom','start','end'])
    pos['chrom'],pos['start'],pos['end'] = chrom, start, end
    pos.to_csv('./tempfile/%s_pos.bed'%(cell_line),sep = '\t',index = False, header = False)
    
    class_threholds = {'23_seq':'CRISPRa_seq','23_epi':'CRISPRa_epi','40_seq':'CRISPRoff_seq','40_epi':'CRISPRoff_epi'}
    
    if 'seq' in model:
            inputs = onehot
            class_threhold = class_threholds[str(bplength)+"_seq"]
            efficiency = predict(inputs, model, class_threhold)
    else:
        epi_data = endo_annotation(chrom = chrom,start= start, end = end, pamCoord = pamCoord, transcript=transcript, gene=gene, cell_line=cell_line, bplength = bplength)
#         endo_annotation(chrom,start, end, pamCoord, transcript, gene, cell_line,bplength)
        class_threhold = class_threholds[str(bplength)+"_epi"]
        inputs = np.concatenate((onehot, epi_data), axis=3)
        
        efficiency = [predict(inputs, model, class_threhold)]
        

    shutil.rmtree('./tempfile')  
    os.mkdir('./tempfile') 
    print("Predicted Phenotype Score:"+str(efficiency[0]),"Probabilities:"+str(efficiency[1]),"Threshold:"+str(efficiency[2]),"classification:"+str(efficiency[3]))


In [19]:
# Only sequence predicted
main_pip(sequence = "GGGCCCAGATCCCTCTATGTGCTCGAAGCAGGTGGACCCC",model = 'CRISPRoff_seq')

checking sequence input...
Predicted Phenotype Score:0.22935502 Probabilities:0.02367191 Threshold:0.2838078 classification:0


In [3]:
# Predicted with epigenetic feature
main_pip(sequence = "GGGCCCAGATCCCTCTATGTGCTCGAAGCAGGTGGACCCC",model = 'CRISPRoff_epi', chrom = "chr12", start = "100592057", end = "100592096",pamCoord = "100592087", transcript = "P1P2",gene = "ACTR6",cell_line="Hek293t")

checking sequence input...
0
0


2022-04-23 03:49:57.310813: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-23 03:49:57.311822: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/mpi/openmpi3-gnu8/3.1.4/lib:/opt/ohpc/pub/compiler/gcc/8.3.0/lib64:/usr/local/cuda/lib64
2022-04-23 03:49:57.311846: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-23 03:49:57.311873: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nfcu01): /proc/driver/nvidia/version does not exist
2022-04-23 03:49:57.312106: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructio

[(0.21985406, -0.0326332, 0.35562915, 0)]

In [ ]:
def Args():
    parser = argparse.ArgumentParser()
    parser.add_argument("-seq", default=None, type=str, help="the input sequence") 
    parser.add_argument("-model", default=None, type=str, help="the path of model path")
    parser.add_argument("-chrom", default=None, type=str, help="the chrom of input sequence")
    parser.add_argument("-start", default=None, type=str, help="the start of input sequence")
    parser.add_argument("-end", default=None, type=str, help="the end of input sequence")
    parser.add_argument("-pamCoord", default=None, type=str, help="the position of pam")
    parser.add_argument("-transcript", default=False, type=str, help="the transcript of inputsequece, please refer our article discription")
    parser.add_argument("-gene", default=None, type=str, help="the gene of input sequence")    
    parser.add_argument("-cell_line", default='Hek293t', type=int, help="The cell line you need")
    #parser.add_argument("-batch", default=None, type=str, help="path to the batch file")
    args = parser.parse_args()

    return args 